# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
y

array([0, 0, 0, ..., 1, 0, 1])

In [6]:
# Define features set X by selecting all columns but credit_ranking
x = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
x.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)



### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
s_scaler = StandardScaler()

# Fit the scaler to the features training dataset
s_scaler.fit(x_train)

# Fit the scaler to the features training dataset
x_train_scaled = s_scaler.transform(x_train)
x_test_scaled = s_scaler.transform(x_test)

x_train.head()
x_test.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
75,8.8,0.41,0.64,2.2,0.093,9.0,42.0,0.99860,3.54,0.66,10.5
1283,8.7,0.63,0.28,2.7,0.096,17.0,69.0,0.99734,3.26,0.63,10.2
408,10.4,0.34,0.58,3.7,0.174,6.0,16.0,0.99700,3.19,0.70,11.3
1281,7.1,0.46,0.20,1.9,0.077,28.0,54.0,0.99560,3.37,0.64,10.4
1118,7.1,0.39,0.12,2.1,0.065,14.0,24.0,0.99252,3.30,0.53,13.3


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [14]:
# Define the the number of inputs (features) to the model
input_features = len(x_train.columns)

# Review the number of features
input_features


11

In [15]:
# Define the number of hidden nodes for the first hidden layer
layer1 = input_features

# Define the number of hidden nodes for the second hidden layer
layer2 = input_features//2

# Define the number of neurons in the output layer
output_layer = 1


In [17]:
# Create the Sequential model instance
seq_model = Sequential()

# Add the first hidden layer
seq_model.add(Dense(layer1, input_dim=input_features, activation='relu'))

# Add the second hidden layer
seq_model.add(Dense(layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
seq_model.add(Dense(output_layer, activation='sigmoid'))

In [19]:
# Display the Sequential model summary
seq_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                132       
                                                                 
 dense_1 (Dense)             (None, 5)                 60        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 198 (792.00 Byte)
Trainable params: 198 (792.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [20]:
# Compile the Sequential model
seq_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [21]:
# Fit the model using 50 epochs and the training data
fit_model = seq_model.fit(x_train_scaled, y_train, epochs=50)


Epoch 1/50
38/38 [==============================] - 1s 3ms/step - loss: 0.7251 - accuracy: 0.5054
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6847 - accuracy: 0.5613
Epoch 3/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6575 - accuracy: 0.6514
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6343 - accuracy: 0.6947
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6129 - accuracy: 0.7239
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5932 - accuracy: 0.7248
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5756 - accuracy: 0.7239
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5607 - accuracy: 0.7323
Epoch 9/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5481 - accuracy: 0.7440
Epoch 10/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5388 - accuracy: 0.7431
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [22]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_acc = seq_model.evaluate(x_test_scaled, y_test, verbose=2)


# Display the model loss and accuracy results
print(f'Loss: {model_loss}, Accuracy: {model_acc}')


13/13 - 0s - loss: 0.5077 - accuracy: 0.7400 - 185ms/epoch - 14ms/step
Loss: 0.5076563358306885, Accuracy: 0.7400000095367432


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [23]:
# Set the model's file path
file_path = 'student_loans.keras'

# Export your model to a keras file
seq_model.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [24]:
# Set the model's file path
file_path = 'student_loans.keras'

# Load the model to a new object
new_model = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [27]:
# Make predictions with the test data
predictions = new_model.predict(x_test_scaled)

# Display a sample of the predictions
predictions[:5]

13/13 [==============================] - 0s 3ms/step


array([[0.4972556 ],
       [0.35515386],
       [0.66122496],
       [0.7706197 ],
       [0.98430187]], dtype=float32)

In [29]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_rounded = pd.DataFrame(predictions.round(), columns=['predictions'])
predictions_rounded


,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [31]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_rounded))


              precision    recall  f1-score   support

           0       0.71      0.75      0.73       188
           1       0.77      0.73      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

  In order to create a recommendation system for student loans you would need a myriad of data, this data would consist of both academic and financial data.

For academic data you would need:
   - GPA
   - Degree/Major
   - School
  
Meanwhile for financial data you would need:
   - History of Loans
   - Payment History
   - Current Employement (if applicable)
   - Expenses
   - Credit Score

You would also need some info from this side of the loan:
   - Loan Amounts
   - Interest Rates
   - Reviews
   - Any Prerequisites

Collecting all of this information would allow students to find the perfect loan that they are looking for as well as making sure that they are eligible to take the loan. It also collects information to make sure that the student is able to see all of the options that they would have so they will not miss any potential loans they could take.

---

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Content-based filtering would be the best fit for the data listed above. This is because of how personal and specific some of this data is to the user, the unique profile created by this data allows for the user to find the perfect loans for them. The effect of this unique profile would be lost with collaborative filtering as it would just be suggesting the options chosen by users with a somewhat similar profile.

---

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

The biggest issues that would arrise from a system like this would be data privacy and fairness.

The problem with collecting all of this data to reccomend the perfect loans for any given student would be that a lot of this data that would need to be collected is highly personal and would be at risk in the case of a security breach.

The second issue that would likely come into effect would be that in order to properly recommend the perfect loans to any given student you would have to make sure that both your model and your data are completely free of any bias. This would not only be a massive task as a system can unintentionally hold bias, but it would also be an incredibly import one as well.